# Automating Portfolio Optimization and Allocation using Python

Siguiendo el ejemplo del siguiente articulo:

https://towardsdatascience.com/automating-portfolio-optimization-using-python-9f344b9380b9

Requiere la libreria PyPortfolio:
`pip install PyPortfolioOpt`

## Importing Libraries

In [1]:
#Importing all required libraries
#Created by Sanket Karve
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_datareader as web
from matplotlib.ticker import FuncFormatter

/home/ruben/miniconda3/envs/ml4t/lib/python3.7/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


### -----> FIX: Comment and add lines

In [2]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.cla import CLA
#from pypfopt.plotting import Plotting # FIX: Comment this line
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices # FIX: Add this line
from matplotlib.ticker import FuncFormatter

## Scrapping Stock and Financial Data from the Web

In [3]:
tickers = ['BSX','AES','BRK-B','SEE','QQQ','SPY']
thelen = len(tickers)
price_data = []
for ticker in range(thelen):
    prices = web.DataReader(tickers[ticker], start='2015-01-01', end = '2020-06-06', data_source='yahoo')
    price_data.append(prices.assign(ticker=ticker)[['Adj Close']])
    df_stocks = pd.concat(price_data, axis=1)

df_stocks.columns=tickers
df_stocks.head()


,BSX,AES,BRK-B,SEE,QQQ,SPY
Date,,,,,,
2015-01-02,13.22,10.845045,149.169998,39.287323,97.849831,183.525574
2015-01-05,13.81,10.536319,147.000000,38.726086,96.414497,180.211166
2015-01-06,13.70,10.298836,146.839996,38.928490,95.121742,178.513748
2015-01-07,14.03,10.314667,148.880005,38.983704,96.347977,180.738266
2015-01-08,14.59,10.472986,151.369995,39.701363,98.192047,183.945465


In [4]:
#Checking if any NaN values in the data
nullin_df = pd.DataFrame(df_stocks,columns=tickers)
print(nullin_df.isnull().sum())

BSX      0
AES      0
BRK-B    0
SEE      0
QQQ      0
SPY      0
dtype: int64


In [5]:
#Annualized Return
mu = expected_returns.mean_historical_return(df_stocks)
#Sample Variance of Portfolio
Sigma = risk_models.sample_cov(df_stocks)

In [6]:
#Max Sharpe Ratio - Tangent to the EF
ef = EfficientFrontier(mu, Sigma, weight_bounds=(-1,1)) #weight bounds in negative allows shorting of stocks
sharpe_pfolio=ef.max_sharpe() #May use add objective to ensure minimum zero weighting to individual stocks
sharpe_pwt=ef.clean_weights()
print(sharpe_pwt)

OrderedDict([('BSX', 0.82826), ('AES', -0.12836), ('BRK-B', -0.87341), ('SEE', -0.47497), ('QQQ', 1.0), ('SPY', 0.64848)])


### ------> FIX: Replace minvol_pwt by sharpe_pwt 

In [7]:
latest_prices = get_latest_prices(df_stocks)
# Allocate Portfolio Value in $ as required to show number of shares/stocks to buy, also bounds for shorting will affect allocation
#Min Volatility Portfolio Allocation $10000
allocation_minv, rem_minv = DiscreteAllocation(sharpe_pwt, latest_prices, total_portfolio_value=10000).lp_portfolio()
print(allocation_minv)
print("Leftover Fund value in$ after building minimum volatility portfolio is ${:.2f}".format(rem_minv))

{'BSX': 87, 'QQQ': 17, 'SPY': 8, 'AES': -17, 'BRK-B': -9, 'SEE': -27}
Leftover Fund value in$ after building minimum volatility portfolio is $30.94
